In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the data into a Pandas DataFrame
market_data_df = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
market_data_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
market_data_df.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
market_data_df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(market_data_df[["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])

market_data_scaled[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [6]:
# Create a DataFrame with the scaled data
market_data_scaled_df = pd.DataFrame(
    market_data_scaled,
    columns=["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d", "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]
)

# Copy the crypto names from the original data
crypto_names = market_data_df.index

# Set the coinid column as index
market_data_scaled_df = market_data_scaled_df.set_index(crypto_names)

# Display sample data
market_data_scaled_df

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [8]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters = i, random_state=1)
    k_model.fit(market_data_scaled_df)
    inertia.append(k_model.inertia_)

In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_1 = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_data_1_df = pd.DataFrame(elbow_data_1)

In [10]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_1 = elbow_data_1_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

elbow_plot_1

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [11]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=1)

In [12]:
# Fit the K-Means model using the scaled data
k_model.fit(market_data_scaled_df)

KMeans(n_clusters=4, random_state=1)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled data
kmeans_predictions = k_model.predict(market_data_scaled_df)

# Print the resulting array of cluster values.
kmeans_predictions

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [14]:
# Create a copy of the DataFrame
market_data_copy_df = market_data_scaled_df.copy()

In [15]:
# Add a new column to the DataFrame with the predicted clusters
market_data_copy_df['prediction'] = kmeans_predictions

# Display sample data
market_data_copy_df

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,prediction
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,2
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,2
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,2
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,2


In [16]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_plot  = market_data_copy_df.hvplot.scatter(x="price_change_percentage_24h", 
                                   y="price_change_percentage_7d", 
                                   by="prediction",
                                   hover_cols='coin_id')

market_data_plot

:NdOverlay   [prediction]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [18]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca.fit_transform(market_data_copy_df)

# View the first five rows of the DataFrame. 
market_data_pca[:5]

array([[ 0.44890795, -1.24537573, -0.8506404 ],
       [ 0.49536709, -0.899823  , -1.31755943],
       [-0.81884571,  0.07189909,  0.69501507],
       [-0.84035711,  0.08005407,  0.54435952],
       [ 0.81324004, -2.66952035, -1.64332113]])

In [19]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_var = pca.explained_variance_ratio_
print(sum(explained_var))

0.8886218549859446


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** *0.8886218549859446* is the total explained variance of the three principal components.

**Create a new DataFrame with the PCA data.**

In [20]:
# Creating a DataFrame with the PCA data
market_data_pca_df = pd.DataFrame(
    market_data_pca,
    columns=["PCA1", "PCA2", "PCA3"]
)

# Copy the crypto names from the original data
market_data_pca_df['coin_id'] = market_data_copy_df.index

# Set the coinid column as index
market_data_pca_df = market_data_pca_df.set_index('coin_id')

# Display sample data
market_data_pca_df.sample(5)


,PCA1,PCA2,PCA3
coin_id,,,
okb,-0.996987,1.292574,0.793701
binancecoin,0.822871,-1.682096,-0.905203
bitcoin,0.448908,-1.245376,-0.850640
bitcoin-cash,0.813240,-2.669520,-1.643321
iota,-1.059844,0.097044,-0.149781


---

### Find the Best Value for k Using the PCA Data

In [21]:
# k-values from 1 to 11
k = list(range(1,11))

In [22]:
# Create a list with the number of k-values from 1 to 11
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters = i, random_state=1)
    k_model.fit(market_data_pca_df)
    inertia.append(k_model.inertia_)

In [23]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_2 = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_data_2_df = pd.DataFrame(elbow_data_2)


In [24]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

elbow_plot_2 = elbow_data_2_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

elbow_plot_2

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k = 4 is the best value


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [25]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=1)

In [26]:
# Fit the K-Means model using the PCA data
k_model.fit(market_data_pca_df)

KMeans(n_clusters=4, random_state=1)

In [27]:
# Predict the clusters to group the cryptocurrencies using the PCA data
kmeans_predictions = k_model.predict(market_data_pca_df)

# Print the resulting array of cluster values.
kmeans_predictions

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0])

In [28]:
# Create a copy of the DataFrame with the PCA data
market_data_pca_copy_df = market_data_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
market_data_pca_copy_df['prediction'] = kmeans_predictions

# Display sample data
market_data_pca_copy_df

,PCA1,PCA2,PCA3,prediction
coin_id,,,,
bitcoin,0.448908,-1.245376,-0.850640,1
ethereum,0.495367,-0.899823,-1.317559,1
tether,-0.818846,0.071899,0.695015,0
ripple,-0.840357,0.080054,0.544360,0
bitcoin-cash,0.813240,-2.669520,-1.643321,1
binancecoin,0.822871,-1.682096,-0.905203,1
chainlink,0.913868,-1.128684,-2.730038,1
cardano,0.811102,-1.113378,-1.598655,1
litecoin,0.258728,-0.946884,-1.103560,1


In [29]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_pca_plot = market_data_pca_copy_df.hvplot.scatter(x="PCA1", 
                                   y="PCA2", 
                                   by="prediction",
                                   hover_cols='coin_id')
market_data_pca_plot

:NdOverlay   [prediction]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [30]:
# Composite plot to contrast the Elbow curves
elbow_plot_1 + elbow_plot_2

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

**Additional Composite Elbow Plot**
---

In [31]:
# Composite plot to contrast the Elbow curves
composite_df = elbow_data_1_df.copy()
composite_df['inertia_pca'] = elbow_data_2_df['inertia']

composite_elbow_plot = composite_df.hvplot.line(
    x="k", 
    y=["inertia", "inertia_pca"], 
    title="Composite Elbow Curve", 
    xticks=k
)

composite_elbow_plot


:NdOverlay   [Variable]
   :Curve   [k]   (value)

In [32]:
market_data_plot + market_data_pca_plot

:Layout
   .NdOverlay.I  :NdOverlay   [prediction]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [prediction]
      :Scatter   [PCA1]   (PCA2,coin_id)

**Additional Composite Cluster Plot**
---

In [33]:
# Composite plot to contrast the clusters
# Prepare Non K-Means Dataframe
market_data_contrast_1 = market_data_copy_df.filter(['price_change_percentage_24h', 
                                                  'price_change_percentage_7d', 
                                                  'prediction'])

market_data_contrast_1 = market_data_contrast_1.rename(columns={'price_change_percentage_24h':'PCA1', 
                                                  'price_change_percentage_7d':'PCA2'})
market_data_contrast_1 = market_data_contrast_1.reset_index()


# Prepare K-Means Dataframe
market_data_contrast_2 = market_data_pca_copy_df.filter(['PCA1', 'PCA2', 'prediction'])
market_data_contrast_2 = market_data_contrast_2.reset_index()


##################################################
# Add marker sizes and labels for the scatter plot
# market_data_contrast_1
marker_size = []
label = []
for i in range(len(market_data_copy_df)):
    marker_size.append(10)
    label.append('Non_K-Means')

market_data_contrast_1['marker_size'] = marker_size
market_data_contrast_1['analysis'] = label

# market_data_contrast_2
marker_size = []
label = []
for i in range(len(market_data_pca_copy_df)):
    marker_size.append(50)
    label.append('K-Means')

market_data_contrast_2['marker_size'] = marker_size
market_data_contrast_2['analysis'] = label
###################################################


# Merge the Dataframes
market_data_frames = [market_data_contrast_1, market_data_contrast_2]
market_data_contrast = pd.concat(market_data_frames)


# Display the plot
market_data_contrast.hvplot.scatter(x="PCA1", 
                                   y="PCA2",
                                   s='marker_size',
                                   scale=1, 
                                   by="prediction",
                                   hover_cols=['coin_id', 'analysis'])

:NdOverlay   [prediction]
   :Scatter   [PCA1]   (PCA2,marker_size,coin_id,analysis)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  ---
  * **Answer:** 
    - The extents of the fewer data clusters are more widely dispersed with the K-Means reduction.
    - Predictions 0 and 1 were changed enough to overlap the eachother between the K-Means and non K-Means. 
    - Prediction 2 was reduced to 1 point distant from the non reduced cluster. 
    - Prediction 3 was separated to opposite ends of the plot.
  ---
    * - Note: When using the overlay of the two plots, the smaller markers represent the non K-Means data. The larger markers show the K-Means data.
  ---
  **Summary** - 
  From this visual, the impact was somewhat significant. It shows that we gained points in the prediction group 1 and lost points in prediction group 2. I would want to see more data to find confidence in this K-Means prediction.